In [ ]:
%matplotlib inline
from astropy.io import fits
from astropy import coordinates as c
import matplotlib.pyplot as plt
import numpy as np
import sqlite3 as sql
import barycorrpy
import csv

In [ ]:
data_dir = 'data/radial_velocity/'

rv_data = {"Anderson_2010_WASP17.csv": {"time": "BJD", "time_offset": 2450000,
                "target": "Wasp-17", "instrument": "multi",},
           "Bayliss_2010_WASP17.csv": {"time": "HJD", "time_offset": 0,
                "columns": ["","",""]},
           "apj493234t3_mrt.txt": {"time": "BJDTDB", "time_offset": 0, 
                "instrument": "HIRES",
                "columns": ["time","rv","rv_err", "target"]},
           "J_A+A_524_A25/rvdata.dat": {"time": "BJD", "time_offset": 0,
                "columns": ["target","instrument","time","rv","rv_err"]}
          }

In [ ]:
# Do the simplest one first

conn = sql.connect('shel_database.sqlite')
cur = conn.cursor()

with open(data_dir + 'apj493234t3_mrt.txt', 'r') as f:
    freader = csv.reader(f, delimiter = ' ')
    for row in freader:
        if row[0][0] == "#":
            continue
        # deal with variable whitespace
        data = [x for x in row if x != ""]
        target_id = cur.execute(f'select id from targets where name = "{data[3]}"').fetchall()
        if target_id == []:
            continue
        target_id = target_id[0][0]
        # This is data ref 1, need to code generalized ref_id retrieval
        stmt = ("insert into radial_velocities (target_id, reference_id, "
                f"instrument, bjd, rv, rv_err) values ({target_id}, 1, 5, {data[0]},"
                f"{data[1]}, {data[2]});")
        #cur.execute(stmt)
        
conn.commit()
cur.close()
conn.close()

In [ ]:
# RV from discovery paper

conn = sql.connect('shel_database.sqlite')
cur = conn.cursor()

# Single target file so we don't need to do this in the loop
target_id = cur.execute(f'select id from targets where name = "WASP-17"').fetchall()
target_id = target_id[0][0]
print(f"Target ID: {target_id}")

# Insert the reference into the database if needed, and/or get ref ID
url = "https://iopscience.iop.org/article/10.1088/0004-637X/709/1/159"
fname = data_dir + "Anderson_2010_WASP17.csv"
refnum = cur.execute(f"select id from data_refs where url='{url}'").fetchone()
if refnum is None:
    stmt = f"insert into data_refs (local_filename, url) values ('{fname}', '{url}')"
    print(stmt)
    cur.execute(stmt)
    refnum = cur.execute(f"select id from data_refs where url='{url}'").fetchone()
elif isinstance(refnum, (list, tuple)):
    refnum = refnum[0]
    
print(f"Refnum: {refnum}")

with open(data_dir + 'Anderson_2010_WASP17.csv', 'r') as f:
    freader = csv.reader(f, delimiter = "\t")
    instrument = "CORALIE"
    switch_inst = False
    for row in freader:
        if row[0][0] == "#":
            if not switch_inst:
                continue
            else:
                instrument = row[0][1:]
                print(f"Switching to {instrument}")
                continue
        else:
            # Next time we encounter a # after the header is a new instrument
            switch_inst = True    
            
            # deal with variable whitespace
            #data = [x for x in row if x != ""]
            data = [float(x) for x in row if x != ""]
        
        # Get instrument_id
        inst_id = cur.execute(f'select id from instruments where name = "{instrument}"').fetchall()
        inst_id = inst_id[0][0]
        print(f"Instrument ID: {inst_id}")
        
        # Convert to m/s from km/s, offset is from paper
        rv = (data[1] + 49.5128)*1000
        rv_err = data[2]*1000
        # Need to code generalized ref_id retrieval
        stmt = ("insert into radial_velocities (target_id, reference_id, "
                f"instrument, bjd, rv, rv_err) values ({target_id}, {refnum}, {inst_id},"
                f"{data[0]+2450000}, {rv}, {rv_err});")
        print(stmt)
        #cur.execute(stmt)
        
conn.commit()
cur.close()
conn.close()


In [ ]:
# Next one needs to convert HJD to BJD
# From https://gist.github.com/StuartLittlefair/4ab7bb8cf21862e250be8cb25f72bb7a
# with some minor edits to the coordinate handling
from astropy.coordinates import SkyCoord, EarthLocation
from astropy import units as u
from astropy.time import Time


def helio_to_bary(coords, hjd, obs_name):
    ra = coords[0].split(":")
    ra = int(ra[0]) + int(ra[1])/60 + float(ra[2])/3600
    dec = coords[1].split(":")
    if int(dec[0]) < 1:
        dec = int(dec[0]) - int(dec[1])/60 - float(dec[2])/3600
    else:
        dec = int(dec[0]) + int(dec[1])/60 + float(dec[2])/3600
    star = SkyCoord(ra=ra*u.hour, dec=dec*u.deg)
    print(star)
    
    helio = Time(hjd, scale='utc', format='jd')
    obs = EarthLocation.of_site(obs_name)
    ltt = helio.light_travel_time(star, 'heliocentric', location=obs)
    print(f"helio: {helio}")
    print(f"ltt: {ltt}")
    guess = helio - ltt
    # if we assume guess is correct - how far is heliocentric time away from true value?
    delta = (guess + guess.light_travel_time(star, 'heliocentric', obs)).jd  - helio.jd
    print(f"delta: {delta}")
    # apply this correction
    guess -= delta * u.d
    print(delta*u.d)
    print(guess.tdb)
    
    ltt = guess.light_travel_time(star, 'barycentric', obs)
    print(f"bary_ltt: {ltt}")
    return guess.tdb + ltt

In [ ]:
conn = sql.connect('shel_database.sqlite')
cur = conn.cursor()

# Single target file so we don't need to do this in the loop
target_id = cur.execute(f'select id from targets where name = "WASP-17"').fetchall()
target_id = target_id[0][0]
print(f"Target ID: {target_id}")

# Get coordinates of target
coords = cur.execute(f'select ra, dec from targets where id = {target_id}').fetchone()
print(f"Coords: {coords}")

# Get site name for instrument
observatory = cur.execute(f"select sitename from instruments where id = 5").fetchone()[0]

# Insert the reference into the database if needed, and/or get ref ID
url = "https://iopscience.iop.org/article/10.1088/2041-8205/722/2/L224"
fname = data_dir + "Bayliss_2010_WASP17.csv"

refnum = cur.execute(f"select id from data_refs where url='{url}'").fetchone()
if refnum is None:
    stmt = f"insert into data_refs (local_filename, url) values ('{fname}', '{url}')"
    print(stmt)
    cur.execute(stmt)
    refnum = cur.execute(f"select id from data_refs where url='{url}'").fetchone()
elif isinstance(refnum, (list, tuple)):
    refnum = refnum[0]
    
print(f"Refnum: {refnum}")

with open(data_dir + 'Bayliss_2010_WASP17.csv', 'r') as f:
    freader = csv.reader(f, delimiter = "\t")
    instrument = "MIKE"
    # Get instrument_id
    inst_id = cur.execute(f'select id from instruments where name = "{instrument}"').fetchall()
    inst_id = inst_id[0][0]
    print(f"Instrument ID: {inst_id}")

    for row in freader:
        if row[0][0] == "#":
            continue

        # deal with variable whitespace
        #data = [x for x in row if x != ""]
        data = [float(x) for x in row if x != ""]
        
        # Convert times from HJD to BJD
        time_bjd = helio_to_bary(coords, data[0], observatory)
        print(data[0] - time_bjd.value)
        rv = data[1]
        rv_err = data[2]
        # Need to code generalized ref_id retrieval
        stmt = ("insert into radial_velocities (target_id, reference_id, "
                f"instrument, bjd, rv, rv_err) values ({target_id}, {refnum}, {inst_id}, "
                f"{time_bjd}, {rv}, {rv_err});")
        #cur.execute(stmt)
        
conn.commit()
cur.close()
conn.close()

In [ ]:
# Last one for WASP-17! In BJD, but need to strip "b" from 
# target names and filter out instruments we don't want

# I can only find the RV offset in the paper for WASP-17, not 6 or 18, so
# I'm only going to load 17 for now.

conn = sql.connect('shel_database.sqlite')
cur = conn.cursor()


# Insert the reference into the database if needed, and/or get ref ID
url = "https://www.aanda.org/articles/aa/full_html/2010/16/aa14525-10/aa14525-10.html"
fname = data_dir + "J_A+A_524_A25/rvdata.dat"
instruments = {"C1": "CORALIE", "H1": "HARPS"}

refnum = cur.execute(f"select id from data_refs where url='{url}'").fetchone()
if refnum is None:
    stmt = f"insert into data_refs (local_filename, url) values ('{fname}', '{url}')"
    cur.execute(stmt)
    refnum = cur.execute(f"select id from data_refs where url='{url}'").fetchone()[0]
elif isinstance(refnum, (list, tuple)):
    refnum = refnum[0]
    
print(f"Refnum: {refnum}")

with open(fname, 'r') as f:
    freader = csv.reader(f, delimiter = ' ')
    for row in freader:
        if row[0][0] == "#":
            continue
            
        # deal with variable whitespace
        data = [x for x in row if x != ""]
        
        # get target ID
        target_id = cur.execute(f'select id from targets where name = "{data[0][:-1]}"').fetchall()
        if target_id == []:
            continue
            
        target_id = target_id[0][0]
        # Skipping all but WASP-17 for now
        if target_id != 42:
            continue
        
        # Get instrument_id. Don't care about Rossiter-McLaughlin effect measurements
        if data[1] in instruments:
            instrument = instruments[data[1]]
        else:
            continue
            
        inst_id = cur.execute(f'select id from instruments where name = "{instrument}"').fetchall()
        inst_id = inst_id[0][0]
        print(f"Instrument ID: {inst_id}")
        
        # Convert to m/s from km/s, offset is from paper
        if inst_id == 4:
            offset = -49.51367
        elif inst_id == 3:
            offset = -49.49168
        else:
            raise ValueError("What instrument is this??")
            
        rv = (float(data[3]) - offset)*1000
        rv_err = float(data[4])*1000
        
        stmt = ("insert into radial_velocities (target_id, reference_id, "
                f"instrument, bjd, rv, rv_err) values ({target_id}, {refnum}, "
                f"{inst_id}, {data[2]}, {rv}, {rv_err});")
        print(stmt)
        #cur.execute(stmt)
        
conn.commit()
cur.close()
conn.close()

In [ ]:
helio_to_bary(('15:59:50.9491505016', '00:03:42.312819096'), 2454336.7382708, 'Keck Observatory')

In [ ]:
from utils import ingest_rv_data